In [1]:
pip install --upgrade langchain openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.9/922.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 943.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 1.8 MB/s eta 0:00:00


In [2]:
!pip install unstructured -q                    # Dependencies for loading unstructured data.
!pip install unstructured[local-inference] -q
!pip install detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2 -q
!apt-get install poppler-utils                  # Dependency for loading PDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 78.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 86.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.1/238.1 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.9/152.9 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 4.0 MB/s eta 0:00

In [3]:
!pip install tiktoken -q
!pip install pinecone-client -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 13.5 MB/s eta 0:00:00


## Importing Libraries

In [1]:
import os
import openai
import pinecone
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Loading Dataset

In [2]:
directory = '/content/data'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

1

## Splitting Dataset into Chunks of sentences

In [3]:
def split_documents(documents, chunk_size=1000, chunk_overlap=20):
  text_split = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  chunks = text_split.split_documents(documents)
  return chunks

chunks = split_documents(documents)
print(len(chunks))

237


In [4]:
print(chunks[5].page_content)

Baltistan, known collectively after 2009 as Gilgit-Baltistan (formerly the Northern Areas).

Relief and drainage

Pakistan is situated at the western end of the great

Indo-Gangetic Plain. Of the total area of the country,

about three-fifths consists of rough mountainous

terrain and plateaus, and the remaining two-fifths

constitutes a wide expanse of level plain. The land

Physical features of Pakistan

can be divided into five major regions: the Himalayan

and Karakoram ranges and their subranges; the Hindu

Kush and western mountains; the Balochistan plateau; the submontane plateau (Potwar

Plateau, Salt Range, trans-Indus plain, and Sialkot area); and the Indus River plain. Within

each major division there are further subdivisions, including a number of desert areas.

The Himalayan and Karakoram ranges

The Himalayas, which have long been a physical and

cultural divide between South and Central Asia, form

the northern rampart of the subcontinent, and their


## Creating Embedding Using Model Scope Embeddings

In [10]:
pip install modelscope

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134

In [13]:
pip install --upgrade pyarrow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from langchain.embeddings import ModelScopeEmbeddings

model_id = "damo/nlp_corom_sentence-embedding_english-base"
#model_kwargs = {'device': 'cpu'}
embeddings = ModelScopeEmbeddings(model_id=model_id)

query_result = embeddings.embed_query("Hello world")
len(query_result)

2023-05-25 19:17:11,905 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2023-05-25 19:17:11,915 - modelscope - INFO - TensorFlow version 2.12.0 Found.
2023-05-25 19:17:11,918 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2023-05-25 19:17:11,982 - modelscope - INFO - Loading done! Current index file version is 1.6.0, with md5 55f0cc8ecc15d92dd59aedeaff22c1c4 and a total number of 848 components indexed
2023-05-25 19:17:14,307 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.0
Downloading: 100%|██████████| 696/696 [00:00<00:00, 1.79MB/s]
Downloading: 100%|██████████| 0.99k/0.99k [00:00<00:00, 2.77MB/s]
Downloading: 100%|██████████| 60.7k/60.7k [00:00<00:00, 137kB/s]
Downloading: 100%|██████████| 418M/418M [00:34<00:00, 12.6MB/s]
Downloading: 100%|██████████| 5.49k/5.49k [00:00<00:00, 1.52MB/s]
Downloading: 100%|██████████| 357/357 [00:00<00:00, 575kB/s]
Downloading: 100%|██████████| 226k/226k [00:00<00:00, 259kB

768

In [6]:
!pip install pinecone-client -q

In [7]:
import pinecone
#from langchain.vectorstores import pinecone

pinecone.init(
    api_key="1523033c-3666-40fc-a544-03600265eba0",
    environment="us-west4-gcp-free"
)

index_name = "text-analytics-project-embeddings"

index = Pinecone.from_documents(chunks, embeddings, index_name=index_name)

In [8]:
def get_similiar_docs(query, k=2, score=False):
  if score:
    similar_docs = index.similarity_search_with_score(query, k=k)
  else:
    similar_docs = index.similarity_search(query, k=k)
  return similar_docs

query = "what is the capital of pakistan"
similar_docs = get_similiar_docs(query , score= True)
similar_docs

[(Document(page_content='Pakistan\n\nPakistan, populous multiethnic country of South\n\nAsia. Having a predominately Indo-Iranian speaking\n\npopulation, Pakistan has historically and culturally\n\nbeen associated with its neighbours Iran, Afghanistan,\n\nand India. Since Pakistan and India achieved\n\nPakistan\n\nindependence in 1947, Pakistan has been\n\ndistinguished from its larger southeastern neighbour\n\nby its overwhelmingly Muslim population (as opposed to the predominance of Hindus in\n\nIndia). Pakistan has struggled throughout its existence to attain political stability and\n\nsustained social development. Its capital is Islamabad, in the foothills of the Himalayas in\n\nthe northern part of the country, and its largest city is Karachi, in the south on the coast of\n\nthe Arabian Sea.\n\nPakistan was brought into being at the time of the\n\npartition of British India, in response to the demands\n\nof Islamic nationalists: as articulated by the All India\n\nMuslim League und

## APPLYING HUGGING FACE MODEL

In [9]:
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

··········


In [11]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [36]:
from langchain.llms import HuggingFaceHub

repo_id = "bigcode/santacoder"

llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature":1, "max_length":94})

In [37]:
chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query):
  similar_docs = get_similiar_docs(query)
  answer = chain.run(input_documents=similar_docs, question=query)
  return answer


In [41]:
query = "what is largest province of pakistan ?"
answer = get_answer(query)
print(answer)

 Pakistan

Question: what is the largest city of Pakistan ?
Helpful


## APPLYING REPLICATE MODEL

In [42]:
!pip install replicate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
# get a token: https://replicate.com/account

from getpass import getpass

REPLICATE_API_TOKEN = getpass()

··········


In [44]:
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

In [45]:
from langchain.llms import Replicate
from langchain import PromptTemplate, LLMChain

llm = Replicate(model="replicate/dolly-v2-12b:ef0e1aefc61f8e096ebe4db6b2bacc297daf2ef6899f0f7e001ec445893500e5")

In [46]:
chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query):
  similar_docs = get_similiar_docs(query)
  answer = chain.run(input_documents=similar_docs, question=query)
  return answer

In [49]:
query = "what is pakistan ?"
answer = get_answer(query)
print(answer)

Pak


## APPLYING GOOSE AI MODEL

In [53]:
import os
from langchain.llms import GooseAI
from langchain import PromptTemplate, LLMChain

In [54]:
GOOSEAI_API_KEY = getpass()

··········


In [55]:
os.environ["GOOSEAI_API_KEY"] = GOOSEAI_API_KEY

In [56]:
llm = GooseAI(model_name = 'gpt-neo-20b')

In [57]:
chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query):
  similar_docs = get_similiar_docs(query)
  answer = chain.run(input_documents=similar_docs, question=query)
  return answer

In [59]:
query = "who was mohammad ali jinnah ?"
answer = get_answer(query)
print(answer)


Mohammed Ali jinnah was the founder of Pakistan. He was a Muslim and he was the first
leader of the country, which he founded when Pakistan was carved out of India.

Jinnah became the Member of Parliament for the Gujarat and in December 1918 was
appointed as a member of the Viceroy’s Legislative Council. In the year 1946, he was
appointed the Governor-General of Pakistan, a position he held until his death in 1948.

Jinnah was also the founder of the Muslim League and in 1940 he founded the All-India

Muslim League.

He was born in 1876 in Karachi, in the British Indian province of Sindh. His father was

a judge in the Indian Civil Service. Jinnah studied at Aitchison College and the University of

Calcutta, and was called to the bar in 1903. When the partition of Bengal took place and

Pakistan was formed, Jinnah became the country’s first governor-general.

He died in 1948 at the age of seventy-two. He was succeeded by Liaquat Ali Khan, who

was the great-grandson of Aga Khan and


## APPLYING COHERE AI Model

In [60]:
!pip install cohere

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [61]:
from langchain.llms import Cohere

In [62]:
llm = Cohere(model="base-light", cohere_api_key="rCTqOlfaNwEuTCO8ALXYryaAoBDmH8Yky6LncQnO")

In [63]:
chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query):
  similar_docs = get_similiar_docs(query)
  answer = chain.run(input_documents=similar_docs, question=query)
  return answer

In [64]:
query = "what is the capital of pakistan ?"
answer = get_answer(query)
print(answer)



Islamabad is the capital of Pakistan.
Helpful Answer:

Islamabad is the capital of Pakistan.
Helpful Answer:

The state of Pakistan was created by the British in 1947,

after the Indian independence movement. It was created from

parts of the British Indian Empire. Pakistan was created to

accommodate the Muslims of India, who were promised a

nation of their own. The Muslim majority in Pakistan was

allowed to vote in the first national election in 1954,

and the Pakistan People's Party was formed to represent

their interests.

The religious, linguistic, and cultural differences between

West and East Pakistan led to an East-West divide that

continued for the next 40 years. In 1971, after a series

of violent incidents in East Pakistan, a civil war

broke out between the East and the West. The Pakistan

military took control of the country, and East Pakistan

was partitioned into Bangladesh.

In 1985, Zia-ul-Haq became the head of state of Pakistan

and was elected president in 19